<a href="https://colab.research.google.com/github/funway/nid-imbalance-study/blob/main/utils/utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 通用函数封装
🚀 NYIT 880 | 🧑🏻‍💻 funway

- **该文件只定义 全局常量，函数 或 类** ‼️
- 在其他 ipynb 文件中运行 `%run full_path/file_name.ipynb` 即可导入该文件中的函数与类
- 由于 Google Drive 的写入缓存, 所以修改该文件后, 可能需要等待一定时间(几十秒), 在引用处才会生效

In [ ]:
utility_version = '1.0.1'
print(f'导入 utility.ipynb 模块. version {utility_version}')

导入 utility.ipynb 模块. version 1.0.1


## 常量

In [ ]:
"""
原始训练集数据分布
X_train shape: (3797547, 70)
y_train labels: {0: 1600000, 1: 228953, 2: 489, 3: 184, 4: 548809, 5: 1384, 6: 460953, 7: 33206, 8: 369530, 9: 111912, 10: 8792, 11: 154683, 12: 128511, 13: 70, 14: 150071}
"""

## 计划尝试三种样本分布模式
resample_schemes = {
    # 模式1. (标签0:非0标签总和) ≈ (160:157); 非0标签按大概比例增强
    1: {
        0: 1600000, 1: 200000, 2: 20000, 3: 20000, 4: 200000,
        5: 20000, 6: 200000, 7: 100000, 8: 200000, 9: 111912,
        10: 50000, 11: 154683, 12: 128511, 13: 20000, 14: 150071
        },

    # 模式2. 最终结果极少数类都有 2% 的占比
    2: {
        0: 800000, 1: 200000, 2: 50000, 3: 50000, 4: 200000,
        5: 50000, 6: 200000, 7: 100000, 8: 200000, 9: 110000,
        10: 50000, 11: 150000, 12: 120000, 13: 50000, 14: 150000
        },

    # 模式3. 基于模式2, 额外删除标签0与中位数类(9, 11, 12)
    3: {
        0: 400000, 1: 200000, 2: 50000, 3: 50000, 4: 200000,
        5: 50000, 6: 200000, 7: 100000, 8: 200000, 9: 100000,
        10: 50000, 11: 100000, 12: 100000, 13: 50000, 14: 150000
        },

    # 模式4. 基于模式1, 额外删除一些质量不好的中位数类(9, 12)
    4: {
        0: 1600000, 1: 200000, 2: 20000, 3: 20000, 4: 200000,
        5: 20000, 6: 200000, 7: 100000, 8: 200000, 9: 100000,
        10: 50000, 11: 154683, 12: 100000, 13: 20000, 14: 150071
        },

    # 模式5. 基于模式1, 使用 CGAN 增加一些质量不好的中位数类(9, 12)，让他在分类训练中比重增大。？
    5: {
        0: 1600000, 1: 200000, 2: 20000, 3: 20000, 4: 200000,
        5: 20000, 6: 200000, 7: 100000, 8: 200000, 9: 160000,
        10: 50000, 11: 154683, 12: 200000, 13: 20000, 14: 150071
        },

    # 模式6. 基于模式2，额外增加中位数类 9, 12
    6: {
        0: 800000, 1: 200000, 2: 50000, 3: 50000, 4: 200000,
        5: 50000, 6: 200000, 7: 100000, 8: 200000, 9: 200000,
        10: 50000, 11: 150000, 12: 200000, 13: 50000, 14: 150000
        },

    # 模式7. 只增加初始识别率低的类
    7: {
        3: 20_000, 9: 200_000, 11: 200_000,
        12: 200_000, 13: 20_000,
    }

}

ros_schemes = {
    1: {2: 1000, 3: 500, 13: 500},
    2: {2: 2000, 3: 1000, 13: 1000, 5: 2000},
}

cgan_filter_schemes = {
    1: {9: 80_000, 12: 80_000},
    2: {9: 100_000, 12: 110_000},
    3: {9: 110_000, 12: 120_000},
}

## 函数

In [ ]:
import numpy as np
from collections import Counter
from io import StringIO

def get_label_counts(y: np.ndarray) -> dict[int, int]:
    """获取标签向量 y 中每个类别样本数量"""
    return {int(k): v for k, v in sorted(Counter(y).items())}

def now() -> str:
    """获取当前时间"""
    from datetime import datetime
    from zoneinfo import ZoneInfo
    return datetime.now(tz=ZoneInfo('America/Vancouver')).strftime('%x %X %Z')


def is_run_by_run_magic() -> bool:
    """
    判断当前 ipynb 文件是否被 %run 调用

    Returns:
        bool: True 表示被 %run 调用, False 表示直接运行
    """
    try:
        _ = __file__  # 尝试访问 __file__ 变量
        return True   # 能访问说明当前脚本是被其他代码文件使用 %run 运行的
    except NameError:
        return False  # 未定义则说明当前脚本是直接运行的

## 测试

In [ ]:
if __name__ == "__main__":
  # print('main guard 在 Jupyter Notebook 中是否有效?')

  # ⚠️ 无效 ⚠️
  # 不论是直接运行 还是在其他文件中使用 %run 运行，都会进入 main guard 代码块
  pass

In [ ]:
if not is_run_by_run_magic():
  print('=== 在该判断条件下的代码将不会被 %run 执行 ===')

  # 在这里写测试代码是 OK 的

  pass

=== 在该判断条件下的代码将不会被 %run 执行 ===
